# 웹캠 연동해서 만든 SEnow

In [5]:
import cv2
import mediapipe as mp
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

def overlay(image, x, y, w, h, overlay_image): # 대상 이미지, x, y 좌표, width, height, 덮어씌울 이미지
    alpha = overlay_image[:, :, 3] #BGRA, A값을 가져옴
    mask_image = alpha / 255 # 0~255 ->255로 나누면 0~1의 값을 가짐, 1: 불투명, 0: 투명
    
    # 얼굴이 창 크기를 벗어나면 오류가 생기므로 예외처리
    try:
        for c in range(0, 3): #BGR 처리
            image[y-h: y+h, x-w: x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h: y+h, x-w: x+w, c] * (1-mask_image))
    except:
        pass

#이미지 불러오기
image_right_eye = cv2.imread('right_eye_cutout.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('left_eye_cutout.png', cv2.IMREAD_UNCHANGED)
image_nose_tip = cv2.imread('nose_tip_cutout.png', cv2.IMREAD_UNCHANGED)

# For static images:
IMAGE_FILES = []
with mp_face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5) as face_detection:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(
            cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            continue
        annotated_image = image.copy()
        for detection in results.detections:
            print('Nose tip:')
            print(mp_face_detection.get_key_point(
                detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
            mp_drawing.draw_detection(annotated_image, detection)
        cv2.imwrite('/tmp/annotated_image' +
                    str(idx) + '.png', annotated_image)

# For webcam input:
cap = cv2.VideoCapture(0)

# 창 크기 출력
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(w, h) # 640, 480

with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
                
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

#         # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                pass
#                 mp_drawing.draw_detection(image, detection)
                
        #특정 위치 가져오기
        keypoints = detection.location_data.relative_keypoints
        right_eye = keypoints[0] # 오른쪽 눈
        left_eye = keypoints[1] # 왼쪽 눈
        nose_tip = keypoints[2] # 코 끝 부분
        
        # 이미지 위치 지정
        right_eye = (int(right_eye.x * w)-20, int(right_eye.y * h)-100)
        left_eye = (int(left_eye.x * w)+20, int(left_eye.y * h)-100) 
        nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h)+30)
        
        # 이미지 대입
        overlay(image, *right_eye, 25, 25, image_right_eye)
        overlay(image, *left_eye, 25, 25, image_left_eye)
        overlay(image, *nose_tip, 50, 50, image_nose_tip)
        
        # Flip the image horizontally for a selfie-view display.
#         cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
        cv2.imshow('MediaPipe Face Detection', cv2.resize(cv2.flip(image, 1), None, fx=1.5, fy=1.5))
        if cv2.waitKey(1) == ord('q'): #q를 누르면 종료
            break
cap.release()
cv2.destroyAllWindows()

640 480
